In [1]:
import geopandas as gpd
import pandas as pd
import os
import getpass
import matplotlib.pyplot as plt

### NOTES:


###### Bring in necessary geojson files and set your projection for all files

In [2]:
wd = os.getcwd()
wd = wd[:wd.find('notebooks')]

#crs is set for Central Texas; https://epsg.io/6578; 
crs =  {'init' :'epsg:6578'}

#parks = gpd.read_file(wd+"data/coaparks/parkboundaries.geojson")
#parks = parks.to_crs(crs).explode().reset_index()

varibuff = gpd.read_file(wd+"data/coaparks_buffer/varibuff.shp")
varibuff.crs={'init' :'epsg:6578'}

pop = gpd.read_file(wd+"data/blockgroups_censusdata/popmerge.shp")
pop = pop.to_crs(crs).reset_index()

race = gpd.read_file(wd+"data/blockgroups_censusdata/racemerge.shp")
race = race.to_crs(crs).reset_index()

income = gpd.read_file(wd+"data/blockgroups_censusdata/incomemerge.shp")
income = income.to_crs(crs).reset_index()

age = gpd.read_file(wd+"data/blockgroups_censusdata/agemerge.shp")
age = age.to_crs(crs).reset_index()


##### Check your projections
https://geopandas.org/projections.html

In [3]:
#varibuff.crs

In [4]:
#pop.crs

In [5]:
#race.crs

In [6]:
#age.crs

###### Preview the files and clean the data

In [7]:
pop['fullarea_pop'] = pop['geometry'].area
#pop.head()

In [8]:
race['fullarea_race'] = race['geometry'].area
#race.head()

In [9]:
income['fullarea_income'] = income['geometry'].area
#income.head()

In [10]:
age['fullarea_age'] = age['geometry'].area
#age.head()

In [11]:
varibuff['fullarea_buff'] = varibuff['geometry'].area
#varibuff.head()

### <font color=green>Run spatial analysis on the amount of people distributed within a census block group against the varied mile park buffer area and the race census data</font>

In [12]:
rp_intersection = gpd.overlay(race, varibuff, how='intersection') #https://geopandas.org/set_operations.html
rp_intersection['area_intersec'] = rp_intersection['geometry'].area
rp_intersection.to_file(wd+"data/access/rp_intersec_varibuff_ACS17.shp")
rp_intersection

,index,GEOID10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,city_name,jurisdicti,Id,Id2,...,GLOBALID,CREATED_BY,CREATED_DA,MODIFIED_B,MODIFIED_D,Shape__Are,Shape__Len,fullarea_buff,geometry,area_intersec
0,0,482090109011,291268.0,0.0,+30.1776997,-097.9267996,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US482090109011,482090109011,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.265813e+07,27658.870722,1.402638e+08,"POLYGON ((3059553.625047085 10032547.61863554,...",1.468736e+07
1045,171,484530017331,10885641.0,0.0,+30.2063570,-097.9389147,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017331,484530017331,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.265813e+07,27658.870722,1.402638e+08,"POLYGON ((3059327.552925358 10045409.39380444,...",2.935787e-03
1274,225,484530017701,5530.0,0.0,+30.1871290,-097.8937674,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017701,484530017701,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.265813e+07,27658.870722,1.402638e+08,"POLYGON ((3067072.500203767 10039770.36438857,...",2.946075e+07
1279,226,484530017702,20689.0,0.0,+30.1826359,-097.8811620,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017702,484530017702,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.265813e+07,27658.870722,1.402638e+08,"POLYGON ((3067967.557403931 10035989.35879634,...",6.003444e+06
1283,227,484530017703,45198.0,0.0,+30.1846012,-097.8872133,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017703,484530017703,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.265813e+07,27658.870722,1.402638e+08,(POLYGON ((3067972.259910929 10038121.77306515...,5.148760e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,542,484910204083,34157.0,0.0,+30.4530947,-097.7811364,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484910204083,484910204083,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,5.423833e+05,2970.882179,4.383920e+06,"POLYGON ((3099630.448388353 10137407.1344326, ...",2.951552e+05
2670,540,484910204081,9762.0,0.0,+30.4597401,-097.7933842,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484910204081,484910204081,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,9.810471e+04,1492.002528,3.127176e+06,"POLYGON ((3100508.462157759 10143807.72265376,...",2.152489e+06
2682,552,484910205081,636722.0,850.0,+30.4840639,-097.7595422,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484910205081,484910205081,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,6.289057e+05,3819.975953,4.931936e+06,(POLYGON ((3105895.006559243 10149425.98327474...,1.307655e+06
2687,556,484910205101,2092.0,0.0,+30.4950296,-097.7673875,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484910205101,484910205101,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,6.289057e+05,3819.975953,4.931936e+06,"POLYGON ((3105356.685648966 10148942.41788262,...",3.024301e+06


In [13]:
#rp_intersection.crs

In [14]:
#rp_intersection.columns

In [15]:
rp_intersection[['GEOID10','Total_POP','fullarea_race','LOCATION_N','fullarea_buff','area_intersec','geometry']]

,GEOID10,Total_POP,fullarea_race,LOCATION_N,fullarea_buff,area_intersec,geometry
0,482090109011,2205,2.280497e+07,Grey Rock Golf Course,1.402638e+08,1.468736e+07,"POLYGON ((3059553.625047085 10032547.61863554,..."
1045,484530017331,2864,2.935787e-03,Grey Rock Golf Course,1.402638e+08,2.935787e-03,"POLYGON ((3059327.552925358 10045409.39380444,..."
1274,484530017701,8224,5.354446e+07,Grey Rock Golf Course,1.402638e+08,2.946075e+07,"POLYGON ((3067072.500203767 10039770.36438857,..."
1279,484530017702,2185,5.361578e+07,Grey Rock Golf Course,1.402638e+08,6.003444e+06,"POLYGON ((3067967.557403931 10035989.35879634,..."
1283,484530017703,2525,1.176147e+07,Grey Rock Golf Course,1.402638e+08,5.148760e+05,(POLYGON ((3067972.259910929 10038121.77306515...
...,...,...,...,...,...,...,...
2672,484910204083,835,9.487768e+05,Springwoods Neighborhood Park,4.383920e+06,2.951552e+05,"POLYGON ((3099630.448388353 10137407.1344326, ..."
2670,484910204081,1797,1.660166e+07,Forest North Neighborhood Park,3.127176e+06,2.152489e+06,"POLYGON ((3100508.462157759 10143807.72265376,..."
2682,484910205081,3775,3.682212e+07,Davis Spring Special Park,4.931936e+06,1.307655e+06,(POLYGON ((3105895.006559243 10149425.98327474...
2687,484910205101,4695,3.462268e+07,Davis Spring Special Park,4.931936e+06,3.024301e+06,"POLYGON ((3105356.685648966 10148942.41788262,..."


In [16]:
race_clip=rp_intersection.copy().reset_index()

for val in race_clip:

    race_clip['weight'] = race_clip['area_intersec']/ race_clip['fullarea_race']
    
    race_clip['access_pop'] = race_clip['weight'] * race_clip['Total_POP']
    race_clip['access_nonhis'] = race_clip['weight'] * race_clip['Not Hispan']
    race_clip['access_white'] = race_clip['weight'] * race_clip['White; Not']
    race_clip['access_his_lat'] = race_clip['weight'] * race_clip['Hispanic o']

race_clip.head()

,level_0,index,GEOID10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,city_name,jurisdicti,Id,...,Shape__Are,Shape__Len,fullarea_buff,geometry,area_intersec,weight,access_pop,access_nonhis,access_white,access_his_lat
0,0,0,482090109011,291268.0,0.0,+30.1776997,-097.9267996,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US482090109011,...,1.265813e+07,27658.870722,1.402638e+08,"POLYGON ((3059553.625047085 10032547.61863554,...",1.468736e+07,0.644042,1420.112697,1148.970998,1051.076608,271.141699
1,1045,171,484530017331,10885641.0,0.0,+30.2063570,-097.9389147,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017331,...,1.265813e+07,27658.870722,1.402638e+08,"POLYGON ((3059327.552925358 10045409.39380444,...",2.935787e-03,1.000000,2864.000000,2445.000000,2217.000000,419.000000
2,1274,225,484530017701,5530.0,0.0,+30.1871290,-097.8937674,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017701,...,1.265813e+07,27658.870722,1.402638e+08,"POLYGON ((3067072.500203767 10039770.36438857,...",2.946075e+07,0.550211,4524.934913,3892.742523,3211.031147,632.192390
3,1279,226,484530017702,20689.0,0.0,+30.1826359,-097.8811620,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017702,...,1.265813e+07,27658.870722,1.402638e+08,"POLYGON ((3067967.557403931 10035989.35879634,...",6.003444e+06,0.111972,244.657907,233.460749,224.167108,11.197158
4,1283,227,484530017703,45198.0,0.0,+30.1846012,-097.8872133,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017703,...,1.265813e+07,27658.870722,1.402638e+08,(POLYGON ((3067972.259910929 10038121.77306515...,5.148760e+05,0.043777,110.535704,86.589949,64.964350,23.945754


In [17]:
#race_clip.columns

In [18]:
race_calc = race_clip[['LOCATION_N', 'Total_POP','access_pop', 'access_nonhis','access_white', 'access_his_lat','area_intersec','geometry']]
access_data = race_calc.dissolve(by='LOCATION_N',as_index=False, aggfunc='sum')

In [19]:
access_data

,LOCATION_N,geometry,Total_POP,access_pop,access_nonhis,access_white,access_his_lat,area_intersec
0,A.B. Dittmar Neighborhood Park,"POLYGON ((3097425.651927478 10040361.9435798, ...",6628,605.761051,409.438174,331.832011,196.322877,4.116064e+06
1,Adams-Hemphill Neighborhood Park,"POLYGON ((3115627.219398506 10079404.88766938,...",16830,3294.880557,2823.146601,2152.343251,471.733956,6.149953e+06
2,Agave Neighborhood Park,"POLYGON ((3144849.303171726 10081396.82272123,...",4529,805.686532,130.358552,74.925119,675.327980,5.240535e+06
3,Alamo Pocket Park,"POLYGON ((3121188.167724669 10076733.3592437, ...",2888,115.493875,105.121175,59.412007,10.372700,9.592893e+05
4,Alderbrook Pocket Park,"POLYGON ((3127839.866424789 10124813.31840664,...",3241,351.309776,247.539393,206.342036,103.770383,9.737850e+05
...,...,...,...,...,...,...,...,...
285,Wunneburger Neighborhood Park,(POLYGON ((3095156.226719216 10021252.69393206...,4954,22.156153,13.475270,11.462701,8.680883,1.393533e+05
286,Yates Pocket Park,"POLYGON ((3128393.558881322 10059201.38858544,...",7507,181.522781,77.983533,24.206194,103.539248,7.161417e+05
287,Yett Creek Neighborhood Park,"POLYGON ((3117627.349335914 10126937.73575986,...",9070,1327.392602,1164.918725,1031.890626,162.473878,1.600528e+07
288,Zilker Metro Park,(POLYGON ((3112028.695926964 10068779.90760447...,34855,14920.662889,13305.905635,11727.823376,1614.757254,8.501994e+07


##### Combine the park area data to the access data

In [20]:
parks_ = varibuff[['LOCATION_N','ASSET_SIZE']].copy()

In [21]:
parks_

,LOCATION_N,ASSET_SIZE
0,Stratford Overlook Greenbelt,0.070000
1,Highland Neighborhood Park,0.241000
2,Palma Plaza Pocket Park,0.206600
3,Oertli Neighborhood Park,6.000000
4,Decker Tallgrass Prairie Preserve,347.991064
...,...,...
293,Red Bluff Neighborhood Park,7.196000
294,MLK Station Neighborhood Park,7.917000
295,Louis RenÃ© Barrera Indiangrass Wildlife Sanct...,290.623199
296,Garrison District Park,40.000000


In [22]:
access_data = access_data.merge(parks_, on='LOCATION_N')

In [23]:
access_data

,LOCATION_N,geometry,Total_POP,access_pop,access_nonhis,access_white,access_his_lat,area_intersec,ASSET_SIZE
0,A.B. Dittmar Neighborhood Park,"POLYGON ((3097425.651927478 10040361.9435798, ...",6628,605.761051,409.438174,331.832011,196.322877,4.116064e+06,12.690000
1,Adams-Hemphill Neighborhood Park,"POLYGON ((3115627.219398506 10079404.88766938,...",16830,3294.880557,2823.146601,2152.343251,471.733956,6.149953e+06,9.310000
2,Agave Neighborhood Park,"POLYGON ((3144849.303171726 10081396.82272123,...",4529,805.686532,130.358552,74.925119,675.327980,5.240535e+06,16.360000
3,Alamo Pocket Park,"POLYGON ((3121188.167724669 10076733.3592437, ...",2888,115.493875,105.121175,59.412007,10.372700,9.592893e+05,1.820000
4,Alderbrook Pocket Park,"POLYGON ((3127839.866424789 10124813.31840664,...",3241,351.309776,247.539393,206.342036,103.770383,9.737850e+05,1.900000
...,...,...,...,...,...,...,...,...,...
286,Wunneburger Neighborhood Park,(POLYGON ((3095156.226719216 10021252.69393206...,4954,22.156153,13.475270,11.462701,8.680883,1.393533e+05,23.510000
287,Yates Pocket Park,"POLYGON ((3128393.558881322 10059201.38858544,...",7507,181.522781,77.983533,24.206194,103.539248,7.161417e+05,0.933800
288,Yett Creek Neighborhood Park,"POLYGON ((3117627.349335914 10126937.73575986,...",9070,1327.392602,1164.918725,1031.890626,162.473878,1.600528e+07,40.545000
289,Zilker Metro Park,(POLYGON ((3112028.695926964 10068779.90760447...,34855,14920.662889,13305.905635,11727.823376,1614.757254,8.501994e+07,307.849066


##### The population served by parks is nomalized by dividing the population served by the area of the park for which they are being served 

In [24]:
access_data['Normalized_byArea'] = access_data['access_pop']/access_data['ASSET_SIZE']
access_data['Normalized_nonhis'] = access_data['access_nonhis']/access_data['ASSET_SIZE']
access_data['Normalized_white'] = access_data['access_white']/access_data['ASSET_SIZE']
access_data['Normalized_hislat'] = access_data['access_his_lat']/access_data['ASSET_SIZE']

In [25]:
access_data.to_file(wd+"data/access/access_data_race_varibuff_ACS17.shp")
access_data

,LOCATION_N,geometry,Total_POP,access_pop,access_nonhis,access_white,access_his_lat,area_intersec,ASSET_SIZE,Normalized_byArea,Normalized_nonhis,Normalized_white,Normalized_hislat
0,A.B. Dittmar Neighborhood Park,"POLYGON ((3097425.651927478 10040361.9435798, ...",6628,605.761051,409.438174,331.832011,196.322877,4.116064e+06,12.690000,47.735307,32.264632,26.149095,15.470676
1,Adams-Hemphill Neighborhood Park,"POLYGON ((3115627.219398506 10079404.88766938,...",16830,3294.880557,2823.146601,2152.343251,471.733956,6.149953e+06,9.310000,353.907686,303.238088,231.186171,50.669598
2,Agave Neighborhood Park,"POLYGON ((3144849.303171726 10081396.82272123,...",4529,805.686532,130.358552,74.925119,675.327980,5.240535e+06,16.360000,49.247343,7.968127,4.579775,41.279216
3,Alamo Pocket Park,"POLYGON ((3121188.167724669 10076733.3592437, ...",2888,115.493875,105.121175,59.412007,10.372700,9.592893e+05,1.820000,63.458173,57.758887,32.643960,5.699286
4,Alderbrook Pocket Park,"POLYGON ((3127839.866424789 10124813.31840664,...",3241,351.309776,247.539393,206.342036,103.770383,9.737850e+05,1.900000,184.899882,130.283891,108.601071,54.615991
...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,Wunneburger Neighborhood Park,(POLYGON ((3095156.226719216 10021252.69393206...,4954,22.156153,13.475270,11.462701,8.680883,1.393533e+05,23.510000,0.942414,0.573172,0.487567,0.369242
287,Yates Pocket Park,"POLYGON ((3128393.558881322 10059201.38858544,...",7507,181.522781,77.983533,24.206194,103.539248,7.161417e+05,0.933800,194.391498,83.512029,25.922247,110.879469
288,Yett Creek Neighborhood Park,"POLYGON ((3117627.349335914 10126937.73575986,...",9070,1327.392602,1164.918725,1031.890626,162.473878,1.600528e+07,40.545000,32.738750,28.731501,25.450503,4.007248
289,Zilker Metro Park,(POLYGON ((3112028.695926964 10068779.90760447...,34855,14920.662889,13305.905635,11727.823376,1614.757254,8.501994e+07,307.849066,48.467462,43.222173,38.096017,5.245289


#### A dataframe is created with the 'acess_data' geoshapefile. This dataframe is cleaned up and the values are convereted into integers. Finally, we export the dataframe into a csv file and a shapefile. 

In [26]:
df = pd.DataFrame(access_data)

df['Park_Name']=df['LOCATION_N']
df['Total_Pop_Served'] = df['access_pop'].astype(int)
df['Normalized_byArea'] = df['Normalized_byArea'].astype(int)
df['Non_Hispan'] = df['access_nonhis'].astype(int)
df['Normalized_nonhis'] = df['Normalized_nonhis'].astype(int)
df['White'] = df['access_white'].astype(int)
df['Normalized_white'] = df['Normalized_white'].astype(int)
df['Hispan_Latin'] = df['access_his_lat'].astype(int)
df['Normalized_hislat'] = df['Normalized_hislat'].astype(int)

df.head()


,LOCATION_N,geometry,Total_POP,access_pop,access_nonhis,access_white,access_his_lat,area_intersec,ASSET_SIZE,Normalized_byArea,Normalized_nonhis,Normalized_white,Normalized_hislat,Park_Name,Total_Pop_Served,Non_Hispan,White,Hispan_Latin
0,A.B. Dittmar Neighborhood Park,"POLYGON ((3097425.651927478 10040361.9435798, ...",6628,605.761051,409.438174,331.832011,196.322877,4.116064e+06,12.69,47,32,26,15,A.B. Dittmar Neighborhood Park,605,409,331,196
1,Adams-Hemphill Neighborhood Park,"POLYGON ((3115627.219398506 10079404.88766938,...",16830,3294.880557,2823.146601,2152.343251,471.733956,6.149953e+06,9.31,353,303,231,50,Adams-Hemphill Neighborhood Park,3294,2823,2152,471
2,Agave Neighborhood Park,"POLYGON ((3144849.303171726 10081396.82272123,...",4529,805.686532,130.358552,74.925119,675.327980,5.240535e+06,16.36,49,7,4,41,Agave Neighborhood Park,805,130,74,675
3,Alamo Pocket Park,"POLYGON ((3121188.167724669 10076733.3592437, ...",2888,115.493875,105.121175,59.412007,10.372700,9.592893e+05,1.82,63,57,32,5,Alamo Pocket Park,115,105,59,10
4,Alderbrook Pocket Park,"POLYGON ((3127839.866424789 10124813.31840664,...",3241,351.309776,247.539393,206.342036,103.770383,9.737850e+05,1.90,184,130,108,54,Alderbrook Pocket Park,351,247,206,103


Export to Shapefile

In [27]:
access_race_final = df.copy().drop(columns=['access_pop','access_nonhis','access_white','access_his_lat'])
access_race_final = gpd.GeoDataFrame(access_race_final, geometry='geometry')
access_race_final.to_file(wd+"data/access/access_race_final_varibuff_ACS17.shp")
access_race_final.head()

,LOCATION_N,geometry,Total_POP,area_intersec,ASSET_SIZE,Normalized_byArea,Normalized_nonhis,Normalized_white,Normalized_hislat,Park_Name,Total_Pop_Served,Non_Hispan,White,Hispan_Latin
0,A.B. Dittmar Neighborhood Park,"POLYGON ((3097425.651927478 10040361.9435798, ...",6628,4.116064e+06,12.69,47,32,26,15,A.B. Dittmar Neighborhood Park,605,409,331,196
1,Adams-Hemphill Neighborhood Park,"POLYGON ((3115627.219398506 10079404.88766938,...",16830,6.149953e+06,9.31,353,303,231,50,Adams-Hemphill Neighborhood Park,3294,2823,2152,471
2,Agave Neighborhood Park,"POLYGON ((3144849.303171726 10081396.82272123,...",4529,5.240535e+06,16.36,49,7,4,41,Agave Neighborhood Park,805,130,74,675
3,Alamo Pocket Park,"POLYGON ((3121188.167724669 10076733.3592437, ...",2888,9.592893e+05,1.82,63,57,32,5,Alamo Pocket Park,115,105,59,10
4,Alderbrook Pocket Park,"POLYGON ((3127839.866424789 10124813.31840664,...",3241,9.737850e+05,1.90,184,130,108,54,Alderbrook Pocket Park,351,247,206,103


Export to CSV

In [28]:
access_racetable = df.drop(columns=['access_pop','access_nonhis','access_white','access_his_lat', 'geometry'])
access_racetable.to_csv(wd+"data/access/access_table_race_varibuff_ACS17.csv")
access_racetable.head()

,LOCATION_N,Total_POP,area_intersec,ASSET_SIZE,Normalized_byArea,Normalized_nonhis,Normalized_white,Normalized_hislat,Park_Name,Total_Pop_Served,Non_Hispan,White,Hispan_Latin
0,A.B. Dittmar Neighborhood Park,6628,4.116064e+06,12.69,47,32,26,15,A.B. Dittmar Neighborhood Park,605,409,331,196
1,Adams-Hemphill Neighborhood Park,16830,6.149953e+06,9.31,353,303,231,50,Adams-Hemphill Neighborhood Park,3294,2823,2152,471
2,Agave Neighborhood Park,4529,5.240535e+06,16.36,49,7,4,41,Agave Neighborhood Park,805,130,74,675
3,Alamo Pocket Park,2888,9.592893e+05,1.82,63,57,32,5,Alamo Pocket Park,115,105,59,10
4,Alderbrook Pocket Park,3241,9.737850e+05,1.90,184,130,108,54,Alderbrook Pocket Park,351,247,206,103


Descriptive Statistics

In [29]:
Race_access_stats = access_racetable.describe()
Race_access_stats.to_csv(wd+"data/access/access_stats_race_varibuff_ACS17.csv")
Race_access_stats

,Total_POP,area_intersec,ASSET_SIZE,Normalized_byArea,Normalized_nonhis,Normalized_white,Normalized_hislat,Total_Pop_Served,Non_Hispan,White,Hispan_Latin
count,291.000000,2.910000e+02,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000
mean,12277.979381,2.442757e+07,57.693347,619.642612,499.230241,393.151203,119.920962,3613.567010,2316.694158,1740.298969,1296.302405
std,14049.278294,4.558888e+07,172.033176,4825.107430,4550.853650,3555.218627,431.004237,7215.862195,4984.335425,3889.269936,3001.056392
min,458.000000,1.078281e+03,0.068466,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4289.500000,1.784429e+06,3.231201,47.000000,28.500000,18.000000,8.000000,293.500000,173.000000,127.500000,61.000000
50%,7157.000000,4.082415e+06,8.450000,96.000000,58.000000,43.000000,22.000000,724.000000,435.000000,327.000000,214.000000
75%,14736.500000,2.366557e+07,30.284929,187.000000,122.000000,99.000000,54.500000,3271.500000,1851.500000,1316.000000,979.500000
max,112988.000000,3.574396e+08,1885.441896,80170.000000,76317.000000,59302.000000,4102.000000,59213.000000,51319.000000,41480.000000,20733.000000


In [30]:
#access_racetable.hist('');

##  <font color=goldenrod>Run spatial analysis on the amount of people distributed within a census block group against the vari buff mile park buffer area and the income census data</font>

In [31]:
ip_intersection = gpd.overlay(income, varibuff, how='intersection') #https://geopandas.org/set_operations.html
ip_intersection['iarea_intersec'] = ip_intersection['geometry'].area
ip_intersection.to_file(wd+"data/access/ip_intersec_varibuff_ACS17.shp")
ip_intersection

,index,GEOID10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,city_name,jurisdicti,Id,Id2,...,GLOBALID,CREATED_BY,CREATED_DA,MODIFIED_B,MODIFIED_D,Shape__Are,Shape__Len,fullarea_buff,geometry,iarea_intersec
0,0,482090109011,291268.0,0.0,+30.1776997,-097.9267996,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US482090109011,482090109011,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.265813e+07,27658.870722,1.402638e+08,"POLYGON ((3059553.625047085 10032547.61863554,...",1.468736e+07
1045,171,484530017331,10885641.0,0.0,+30.2063570,-097.9389147,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017331,484530017331,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.265813e+07,27658.870722,1.402638e+08,"POLYGON ((3059327.552925358 10045409.39380444,...",2.935787e-03
1274,225,484530017701,5530.0,0.0,+30.1871290,-097.8937674,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017701,484530017701,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.265813e+07,27658.870722,1.402638e+08,"POLYGON ((3067072.500203767 10039770.36438857,...",2.946075e+07
1279,226,484530017702,20689.0,0.0,+30.1826359,-097.8811620,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017702,484530017702,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.265813e+07,27658.870722,1.402638e+08,"POLYGON ((3067967.557403931 10035989.35879634,...",6.003444e+06
1283,227,484530017703,45198.0,0.0,+30.1846012,-097.8872133,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017703,484530017703,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.265813e+07,27658.870722,1.402638e+08,(POLYGON ((3067972.259910929 10038121.77306515...,5.148760e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,542,484910204083,34157.0,0.0,+30.4530947,-097.7811364,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484910204083,484910204083,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,5.423833e+05,2970.882179,4.383920e+06,"POLYGON ((3099630.448388353 10137407.1344326, ...",2.951552e+05
2670,540,484910204081,9762.0,0.0,+30.4597401,-097.7933842,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484910204081,484910204081,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,9.810471e+04,1492.002528,3.127176e+06,"POLYGON ((3100508.462157759 10143807.72265376,...",2.152489e+06
2682,552,484910205081,636722.0,850.0,+30.4840639,-097.7595422,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484910205081,484910205081,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,6.289057e+05,3819.975953,4.931936e+06,(POLYGON ((3105895.006559243 10149425.98327474...,1.307655e+06
2687,556,484910205101,2092.0,0.0,+30.4950296,-097.7673875,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484910205101,484910205101,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,6.289057e+05,3819.975953,4.931936e+06,"POLYGON ((3105356.685648966 10148942.41788262,...",3.024301e+06


In [32]:
#ip_intersection.columns

In [33]:
ip_intersection[['GEOID10','Total_Pop','fullarea_income','LOCATION_N','fullarea_buff','iarea_intersec','geometry']]

,GEOID10,Total_Pop,fullarea_income,LOCATION_N,fullarea_buff,iarea_intersec,geometry
0,482090109011,772,2.280497e+07,Grey Rock Golf Course,1.402638e+08,1.468736e+07,"POLYGON ((3059553.625047085 10032547.61863554,..."
1045,484530017331,960,2.935787e-03,Grey Rock Golf Course,1.402638e+08,2.935787e-03,"POLYGON ((3059327.552925358 10045409.39380444,..."
1274,484530017701,2534,5.354446e+07,Grey Rock Golf Course,1.402638e+08,2.946075e+07,"POLYGON ((3067072.500203767 10039770.36438857,..."
1279,484530017702,640,5.361578e+07,Grey Rock Golf Course,1.402638e+08,6.003444e+06,"POLYGON ((3067967.557403931 10035989.35879634,..."
1283,484530017703,889,1.176147e+07,Grey Rock Golf Course,1.402638e+08,5.148760e+05,(POLYGON ((3067972.259910929 10038121.77306515...
...,...,...,...,...,...,...,...
2672,484910204083,280,9.487768e+05,Springwoods Neighborhood Park,4.383920e+06,2.951552e+05,"POLYGON ((3099630.448388353 10137407.1344326, ..."
2670,484910204081,953,1.660166e+07,Forest North Neighborhood Park,3.127176e+06,2.152489e+06,"POLYGON ((3100508.462157759 10143807.72265376,..."
2682,484910205081,1733,3.682212e+07,Davis Spring Special Park,4.931936e+06,1.307655e+06,(POLYGON ((3105895.006559243 10149425.98327474...
2687,484910205101,1366,3.462268e+07,Davis Spring Special Park,4.931936e+06,3.024301e+06,"POLYGON ((3105356.685648966 10148942.41788262,..."


In [34]:
#ip_intersection.columns

In [35]:
income_clip=ip_intersection.copy().reset_index()

for val in income_clip:

    income_clip['weight'] = income_clip['iarea_intersec']/ income_clip['fullarea_income']
    
    income_clip['access_by_income_TotalPop'] = income_clip['weight'] * income_clip['Total_Pop']
    
    income_clip['Less than $25,000'] = (income_clip['weight'] * income_clip['Less than'])\
                                        +(income_clip['weight'] * income_clip['$10,000 to'])\
                                        +(income_clip['weight'] * income_clip['$15,000 to'])\
                                        +(income_clip['weight'] * income_clip['$20,000 to'])
    
    income_clip['$25,000 to $49,999'] = (income_clip['weight'] * income_clip['$25,000 to'])\
                                        +(income_clip['weight'] * income_clip['$30,000 to'])\
                                        +(income_clip['weight'] * income_clip['$35,000 to'])\
                                        +(income_clip['weight'] * income_clip['$40,000 to'])\
                                        +(income_clip['weight'] * income_clip['$45,000 to'])\
    
    income_clip['$50,000 to $74,999'] = (income_clip['weight'] * income_clip['$50,000 to'])\
                                        +(income_clip['weight'] * income_clip['$60,000 to'])\
    
    income_clip['$75,000 to $99,999'] = (income_clip['weight'] * income_clip['$75,000 to'])
    
    income_clip['$100,000 to $149,999'] = (income_clip['weight'] * income_clip['$100,000 t'])\
                                            +(income_clip['weight'] * income_clip['$125,000 t'])\
    
    income_clip['$150,000 or more'] = (income_clip['weight'] * income_clip['$150,000 t'])\
                                        +(income_clip['weight'] * income_clip['$200,000 o'])\
    
income_clip.head()

,level_0,index,GEOID10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,city_name,jurisdicti,Id,...,geometry,iarea_intersec,weight,access_by_income_TotalPop,"Less than $25,000","$25,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 or more"
0,0,0,482090109011,291268.0,0.0,+30.1776997,-097.9267996,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US482090109011,...,"POLYGON ((3059553.625047085 10032547.61863554,...",1.468736e+07,0.644042,497.200454,8.372547,84.369507,63.116120,41.218691,109.487147,190.636444
1,1045,171,484530017331,10885641.0,0.0,+30.2063570,-097.9389147,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017331,...,"POLYGON ((3059327.552925358 10045409.39380444,...",2.935787e-03,1.000000,960.000000,37.000000,67.000000,74.000000,146.000000,245.000000,391.000000
2,1274,225,484530017701,5530.0,0.0,+30.1871290,-097.8937674,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017701,...,"POLYGON ((3067072.500203767 10039770.36438857,...",2.946075e+07,0.550211,1394.234566,18.156962,74.278479,138.653161,139.753583,354.886067,668.506313
3,1279,226,484530017702,20689.0,0.0,+30.1826359,-097.8811620,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017702,...,"POLYGON ((3067967.557403931 10035989.35879634,...",6.003444e+06,0.111972,71.661813,0.000000,4.702806,3.471119,9.741528,18.699254,35.047105
4,1283,227,484530017703,45198.0,0.0,+30.1846012,-097.8872133,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530017703,...,(POLYGON ((3067972.259910929 10038121.77306515...,5.148760e+05,0.043777,38.917323,2.670367,2.407708,1.269519,3.896110,11.381894,17.291724


In [36]:
#income_clip.columns

In [37]:
income_calc = income_clip[['LOCATION_N', 'Total_Pop','access_by_income_TotalPop', 'Less than $25,000','$25,000 to $49,999',\
                           '$50,000 to $74,999','$75,000 to $99,999','$100,000 to $149,999','$150,000 or more','geometry']]

income_access_data = income_calc.dissolve(by='LOCATION_N',as_index=False, aggfunc='sum')

##### Combine the park area data to the access data

In [38]:
income_access_data = income_access_data.merge(parks_, on='LOCATION_N')

In [39]:
income_access_data

,LOCATION_N,geometry,Total_Pop,access_by_income_TotalPop,"Less than $25,000","$25,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 or more",ASSET_SIZE
0,A.B. Dittmar Neighborhood Park,"POLYGON ((3097425.651927478 10040361.9435798, ...",2632,256.533848,27.478335,90.826346,53.930868,30.929280,36.997094,16.371924,12.690000
1,Adams-Hemphill Neighborhood Park,"POLYGON ((3115627.219398506 10079404.88766938,...",3107,1149.515746,580.756475,225.771112,139.505066,101.377739,15.495519,86.609834,9.310000
2,Agave Neighborhood Park,"POLYGON ((3144849.303171726 10081396.82272123,...",1119,188.946324,37.522258,89.655032,25.115956,14.925810,18.224241,3.503027,16.360000
3,Alamo Pocket Park,"POLYGON ((3121188.167724669 10076733.3592437, ...",1201,56.422274,10.489195,12.656636,13.660992,3.467118,12.244006,3.904327,1.820000
4,Alderbrook Pocket Park,"POLYGON ((3127839.866424789 10124813.31840664,...",1484,165.920274,20.371187,46.378518,44.191437,20.649941,29.151265,5.177926,1.900000
...,...,...,...,...,...,...,...,...,...,...,...
286,Wunneburger Neighborhood Park,(POLYGON ((3095156.226719216 10021252.69393206...,1959,8.761385,0.840807,1.788951,1.359602,1.668196,1.990208,1.113622,23.510000
287,Yates Pocket Park,"POLYGON ((3128393.558881322 10059201.38858544,...",2328,55.257182,26.144081,7.275642,8.080470,7.291754,4.750593,1.714643,0.933800
288,Yett Creek Neighborhood Park,"POLYGON ((3117627.349335914 10126937.73575986,...",3953,565.541025,14.367929,80.151240,86.262589,135.990448,114.828700,133.940120,40.545000
289,Zilker Metro Park,(POLYGON ((3112028.695926964 10068779.90760447...,18399,8064.710373,1169.029591,1374.516805,1249.193142,997.626495,1401.337342,1873.006998,307.849066


##### The population served by parks is nomalized by dividing the population served by the area of the park for which they are being served 

In [40]:
income_access_data['TotalPop_Served_norm'] = income_access_data['access_by_income_TotalPop']/income_access_data['ASSET_SIZE']
income_access_data['Less than $25,000_norm'] = income_access_data['Less than $25,000']/income_access_data['ASSET_SIZE']
income_access_data['$25,000 to $49,999_norm'] = income_access_data['$25,000 to $49,999']/income_access_data['ASSET_SIZE']
income_access_data['$50,000 to $74,999_norm'] = income_access_data['$50,000 to $74,999']/income_access_data['ASSET_SIZE']
income_access_data['$75,000 to $99,999_norm'] = income_access_data['$75,000 to $99,999']/income_access_data['ASSET_SIZE']
income_access_data['$100,000 to $149,999_norm'] = income_access_data['$100,000 to $149,999']/income_access_data['ASSET_SIZE']
income_access_data['$150,000 or more_norm'] = income_access_data['$150,000 or more']/income_access_data['ASSET_SIZE']


In [41]:
income_access_data.to_file(wd+"data/access/access_data_income_varibuff_ACS17.shp")
income_access_data

,LOCATION_N,geometry,Total_Pop,access_by_income_TotalPop,"Less than $25,000","$25,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 or more",ASSET_SIZE,TotalPop_Served_norm,"Less than $25,000_norm","$25,000 to $49,999_norm","$50,000 to $74,999_norm","$75,000 to $99,999_norm","$100,000 to $149,999_norm","$150,000 or more_norm"
0,A.B. Dittmar Neighborhood Park,"POLYGON ((3097425.651927478 10040361.9435798, ...",2632,256.533848,27.478335,90.826346,53.930868,30.929280,36.997094,16.371924,12.690000,20.215433,2.165353,7.157317,4.249871,2.437296,2.915453,1.290144
1,Adams-Hemphill Neighborhood Park,"POLYGON ((3115627.219398506 10079404.88766938,...",3107,1149.515746,580.756475,225.771112,139.505066,101.377739,15.495519,86.609834,9.310000,123.471079,62.379858,24.250388,14.984432,10.889123,1.664395,9.302882
2,Agave Neighborhood Park,"POLYGON ((3144849.303171726 10081396.82272123,...",1119,188.946324,37.522258,89.655032,25.115956,14.925810,18.224241,3.503027,16.360000,11.549286,2.293537,5.480136,1.535205,0.912336,1.113951,0.214121
3,Alamo Pocket Park,"POLYGON ((3121188.167724669 10076733.3592437, ...",1201,56.422274,10.489195,12.656636,13.660992,3.467118,12.244006,3.904327,1.820000,31.001249,5.763294,6.954195,7.506040,1.905010,6.727476,2.145235
4,Alderbrook Pocket Park,"POLYGON ((3127839.866424789 10124813.31840664,...",1484,165.920274,20.371187,46.378518,44.191437,20.649941,29.151265,5.177926,1.900000,87.326460,10.721677,24.409746,23.258651,10.868390,15.342771,2.725224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,Wunneburger Neighborhood Park,(POLYGON ((3095156.226719216 10021252.69393206...,1959,8.761385,0.840807,1.788951,1.359602,1.668196,1.990208,1.113622,23.510000,0.372666,0.035764,0.076093,0.057831,0.070957,0.084654,0.047368
287,Yates Pocket Park,"POLYGON ((3128393.558881322 10059201.38858544,...",2328,55.257182,26.144081,7.275642,8.080470,7.291754,4.750593,1.714643,0.933800,59.174537,27.997517,7.791435,8.653319,7.808689,5.087377,1.836200
288,Yett Creek Neighborhood Park,"POLYGON ((3117627.349335914 10126937.73575986,...",3953,565.541025,14.367929,80.151240,86.262589,135.990448,114.828700,133.940120,40.545000,13.948478,0.354370,1.976846,2.127576,3.354062,2.832130,3.303493
289,Zilker Metro Park,(POLYGON ((3112028.695926964 10068779.90760447...,18399,8064.710373,1169.029591,1374.516805,1249.193142,997.626495,1401.337342,1873.006998,307.849066,26.196962,3.797412,4.464905,4.057810,3.240635,4.552027,6.084173


#### A dataframe is created with the 'acess_data' geoshapefile. This dataframe is cleaned up and the values are convereted into integers. Finally, we export the dataframe into a csv file and a shapefile. 

In [42]:
df = pd.DataFrame(income_access_data)
df['Park_Name']=df['LOCATION_N']

df['Total Pop Served'] = df['access_by_income_TotalPop'].astype(int)
df['TotalPop_Served_norm'] = df['TotalPop_Served_norm'].astype(int)

df['Less than $25,000'] = df['Less than $25,000'].astype(int)
df['Less than $25,000_norm'] = df['Less than $25,000_norm'].astype(int)

df['$25,000 to $49,999'] = df['$25,000 to $49,999'].astype(int)
df['$25,000 to $49,999_norm'] = df['$25,000 to $49,999_norm'].astype(int)

df['$50,000 to $74,999'] = df['$50,000 to $74,999'].astype(int)
df['$50,000 to $74,999_norm'] = df['$50,000 to $74,999_norm'].astype(int)

df['$75,000 to $99,999'] = df['$75,000 to $99,999'].astype(int)
df['$75,000 to $99,999_norm'] = df['$75,000 to $99,999_norm'].astype(int)

df['$100,000 to $149,999'] = df['$100,000 to $149,999'].astype(int)
df['$100,000 to $149,999_norm'] = df['$100,000 to $149,999_norm'].astype(int)

df['$150,000 or more'] = df['$150,000 or more'].astype(int)
df['$150,000 or more_norm'] = df['$150,000 or more_norm'].astype(int)


In [43]:
access_income_final = df.copy().drop(columns=['access_by_income_TotalPop'])
access_income_final = gpd.GeoDataFrame(access_income_final, geometry='geometry')
access_income_final.to_file(wd+"data/access/access_income_final_varibuff_ACS17.shp")
access_income_final.head()

,LOCATION_N,geometry,Total_Pop,"Less than $25,000","$25,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 or more",ASSET_SIZE,TotalPop_Served_norm,"Less than $25,000_norm","$25,000 to $49,999_norm","$50,000 to $74,999_norm","$75,000 to $99,999_norm","$100,000 to $149,999_norm","$150,000 or more_norm",Park_Name,Total Pop Served
0,A.B. Dittmar Neighborhood Park,"POLYGON ((3097425.651927478 10040361.9435798, ...",2632,27,90,53,30,36,16,12.69,20,2,7,4,2,2,1,A.B. Dittmar Neighborhood Park,256
1,Adams-Hemphill Neighborhood Park,"POLYGON ((3115627.219398506 10079404.88766938,...",3107,580,225,139,101,15,86,9.31,123,62,24,14,10,1,9,Adams-Hemphill Neighborhood Park,1149
2,Agave Neighborhood Park,"POLYGON ((3144849.303171726 10081396.82272123,...",1119,37,89,25,14,18,3,16.36,11,2,5,1,0,1,0,Agave Neighborhood Park,188
3,Alamo Pocket Park,"POLYGON ((3121188.167724669 10076733.3592437, ...",1201,10,12,13,3,12,3,1.82,31,5,6,7,1,6,2,Alamo Pocket Park,56
4,Alderbrook Pocket Park,"POLYGON ((3127839.866424789 10124813.31840664,...",1484,20,46,44,20,29,5,1.90,87,10,24,23,10,15,2,Alderbrook Pocket Park,165


In [44]:
access_incometable = df.drop(columns=['access_by_income_TotalPop','geometry'])
access_incometable.to_csv(wd+"data/access/access_table_income_varibuff_ACS17.csv")
access_incometable.head()

,LOCATION_N,Total_Pop,"Less than $25,000","$25,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 or more",ASSET_SIZE,TotalPop_Served_norm,"Less than $25,000_norm","$25,000 to $49,999_norm","$50,000 to $74,999_norm","$75,000 to $99,999_norm","$100,000 to $149,999_norm","$150,000 or more_norm",Park_Name,Total Pop Served
0,A.B. Dittmar Neighborhood Park,2632,27,90,53,30,36,16,12.69,20,2,7,4,2,2,1,A.B. Dittmar Neighborhood Park,256
1,Adams-Hemphill Neighborhood Park,3107,580,225,139,101,15,86,9.31,123,62,24,14,10,1,9,Adams-Hemphill Neighborhood Park,1149
2,Agave Neighborhood Park,1119,37,89,25,14,18,3,16.36,11,2,5,1,0,1,0,Agave Neighborhood Park,188
3,Alamo Pocket Park,1201,10,12,13,3,12,3,1.82,31,5,6,7,1,6,2,Alamo Pocket Park,56
4,Alderbrook Pocket Park,1484,20,46,44,20,29,5,1.90,87,10,24,23,10,15,2,Alderbrook Pocket Park,165


In [45]:
access_income_stats = access_incometable.describe()
access_income_stats.to_csv(wd+"data/access/access_stats_income_varibuff_ACS17.csv")
access_income_stats

,Total_Pop,"Less than $25,000","$25,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 or more",ASSET_SIZE,TotalPop_Served_norm,"Less than $25,000_norm","$25,000 to $49,999_norm","$50,000 to $74,999_norm","$75,000 to $99,999_norm","$100,000 to $149,999_norm","$150,000 or more_norm",Total Pop Served
count,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000
mean,4900.305842,248.759450,300.601375,259.474227,180.219931,217.924399,226.230241,57.693347,268.202749,48.013746,41.113402,36.312715,27.848797,40.353952,72.147766,1435.632302
std,5548.564592,586.303832,659.081215,570.864657,372.307149,443.921733,519.898364,172.033176,2020.162499,323.419541,215.829157,207.379217,195.290494,306.082536,796.408634,2902.668950
min,168.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068466,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1780.000000,16.000000,22.500000,17.500000,10.500000,15.500000,11.500000,3.231201,18.000000,2.000000,3.000000,2.000000,1.000000,2.000000,1.000000,119.000000
50%,2810.000000,46.000000,53.000000,50.000000,34.000000,41.000000,42.000000,8.450000,40.000000,5.000000,6.000000,6.000000,3.000000,5.000000,5.000000,297.000000
75%,5655.000000,167.000000,227.000000,217.500000,143.500000,169.000000,146.000000,30.284929,77.500000,15.000000,15.000000,13.500000,9.500000,13.000000,13.500000,1156.000000
max,44300.000000,6161.000000,5651.000000,5545.000000,3141.000000,3337.000000,5190.000000,1885.441896,33068.000000,5170.000000,3256.000000,3135.000000,3107.000000,4931.000000,13467.000000,23211.000000


## <font color=peru>Run spatial analysis on the amount of people distributed within a census block group against the varied park buffer area and the age/sex census data</font>

In [46]:
age_intersection = gpd.overlay(age, varibuff, how='intersection') #https://geopandas.org/set_operations.html
age_intersection['agearea_intersec'] = age_intersection['geometry'].area
age_intersection.to_file(wd+"data/access/age_intersec_varibuff_ACS17.shp")

age_intersection

,index,GEOID10,Id2,Total_pop_,Male,Male_Under,Male_5 to,Male_10 to,Male_15 to,Male_Und_1,...,GLOBALID,CREATED_BY,CREATED_DA,MODIFIED_B,MODIFIED_D,Shape__Are,Shape__Len,fullarea_buff,geometry,agearea_intersec
1,0,484530001011,484530001011,1045,515,72,35,39,30,176,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,3.514350e+06,56641.286637,2.578402e+08,(POLYGON ((3111310.380106392 10089474.08471978...,9.692600e+06
5,1,484530001012,484530001012,1202,599,37,36,18,15,106,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,3.514350e+06,56641.286637,2.578402e+08,"POLYGON ((3109356.370562974 10091521.99532213,...",1.048851e+07
9,2,484530001013,484530001013,2016,797,43,52,23,0,118,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,3.514350e+06,56641.286637,2.578402e+08,"POLYGON ((3110878.498096142 10085236.13059729,...",1.124498e+07
13,3,484530001021,484530001021,1578,734,42,28,90,33,193,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,3.514350e+06,56641.286637,2.578402e+08,"POLYGON ((3108224.684292888 10096983.73086741,...",2.607289e+06
20,4,484530001022,484530001022,1062,492,0,17,44,32,93,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,3.514350e+06,56641.286637,2.578402e+08,"POLYGON ((3109260.211634551 10085726.10829187,...",9.794130e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2669,540,484910204083,484910204083,835,432,73,40,24,0,137,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,5.423833e+05,2970.882179,4.383920e+06,"POLYGON ((3099630.448388353 10137407.1344326, ...",2.951552e+05
2667,538,484910204081,484910204081,1797,904,35,85,47,17,184,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,9.810471e+04,1492.002528,3.127176e+06,"POLYGON ((3100508.462157759 10143807.72265376,...",2.152489e+06
2679,550,484910205081,484910205081,3775,1797,228,89,40,104,461,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,6.289057e+05,3819.975953,4.931936e+06,(POLYGON ((3105895.006559243 10149425.98327474...,1.307655e+06
2684,554,484910205101,484910205101,4695,2365,361,177,191,218,947,...,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,6.289057e+05,3819.975953,4.931936e+06,"POLYGON ((3105356.685648966 10148942.41788262,...",3.024301e+06


In [47]:
age_intersec=age_intersection.drop(['GLOBALID', 'CREATED_BY', 'CREATED_DA', 'MODIFIED_B', 'MODIFIED_D',
       'Shape__Are', 'Shape__Len', 'fullarea_buff','MXASSETNUM', 'MXLOCATION', 'MXSITEID', 'MXCREATION', 'MXSTATUS',
       'MXCONDITIO', 'MXPRIORITY', 'MXLOADID'], axis=1)

In [48]:
age_intersec.columns

Index(['index', 'GEOID10', 'Id2', 'Total_pop_', 'Male', 'Male_Under',
       'Male_5 to', 'Male_10 to', 'Male_15 to', 'Male_Und_1', 'Male_18 an',
       'Male_20 ye', 'Male_21 ye', 'Male_22 to', 'Male_25 to', 'Male_30 to',
       'Male_35 to', 'Male_40 to', 'Male_45 to', 'Male_50 to', 'Male_55 to',
       'Male_60 an', 'Male_62 to', 'Male_65 an', 'Male_67 to', 'Male_70 to',
       'Male_75 to', 'Male_80 to', 'Male85 yea', 'Male_Over6', 'Female_',
       'Female_Und', 'Female_5 t', 'Female_10', 'Female_15', 'Female_U_1',
       'Female_18', 'Estimate;', 'Estimate_1', 'Estimate_2', 'Estimate_3',
       'Estimate_4', 'Estimate_5', 'Estimate_6', 'Estimate_7', 'Estimate_8',
       'Estimate_9', 'Estimate10', 'Estimate11', 'Female_65', 'Female_67',
       'Female_70', 'Female75 t', 'Female80 t', 'Female85 y', 'Female_Ove',
       'fullarea_age', 'ASSET_MGMT', 'LOCATION_N', 'ADDRESS', 'CITY_MUNIC',
       'COUNTY', 'STATE', 'STREET_NUM', 'STREET_NAM', 'STREET_TYP', 'ZIPCODE',
       'ASSET_SI

In [49]:
age_intersection[['GEOID10','Total_pop_','fullarea_age','LOCATION_N','fullarea_buff','agearea_intersec','geometry']]

,GEOID10,Total_pop_,fullarea_age,LOCATION_N,fullarea_buff,agearea_intersec,geometry
1,484530001011,1045,1.023445e+07,Shoal Creek Greenbelt,2.578402e+08,9.692600e+06,(POLYGON ((3111310.380106392 10089474.08471978...
5,484530001012,1202,1.190644e+07,Shoal Creek Greenbelt,2.578402e+08,1.048851e+07,"POLYGON ((3109356.370562974 10091521.99532213,..."
9,484530001013,2016,1.151098e+07,Shoal Creek Greenbelt,2.578402e+08,1.124498e+07,"POLYGON ((3110878.498096142 10085236.13059729,..."
13,484530001021,1578,3.289713e+07,Shoal Creek Greenbelt,2.578402e+08,2.607289e+06,"POLYGON ((3108224.684292888 10096983.73086741,..."
20,484530001022,1062,2.777857e+07,Shoal Creek Greenbelt,2.578402e+08,9.794130e+06,"POLYGON ((3109260.211634551 10085726.10829187,..."
...,...,...,...,...,...,...,...
2669,484910204083,835,9.487768e+05,Springwoods Neighborhood Park,4.383920e+06,2.951552e+05,"POLYGON ((3099630.448388353 10137407.1344326, ..."
2667,484910204081,1797,1.660166e+07,Forest North Neighborhood Park,3.127176e+06,2.152489e+06,"POLYGON ((3100508.462157759 10143807.72265376,..."
2679,484910205081,3775,3.682212e+07,Davis Spring Special Park,4.931936e+06,1.307655e+06,(POLYGON ((3105895.006559243 10149425.98327474...
2684,484910205101,4695,3.462268e+07,Davis Spring Special Park,4.931936e+06,3.024301e+06,"POLYGON ((3105356.685648966 10148942.41788262,..."


In [50]:
age_intersec.columns

Index(['index', 'GEOID10', 'Id2', 'Total_pop_', 'Male', 'Male_Under',
       'Male_5 to', 'Male_10 to', 'Male_15 to', 'Male_Und_1', 'Male_18 an',
       'Male_20 ye', 'Male_21 ye', 'Male_22 to', 'Male_25 to', 'Male_30 to',
       'Male_35 to', 'Male_40 to', 'Male_45 to', 'Male_50 to', 'Male_55 to',
       'Male_60 an', 'Male_62 to', 'Male_65 an', 'Male_67 to', 'Male_70 to',
       'Male_75 to', 'Male_80 to', 'Male85 yea', 'Male_Over6', 'Female_',
       'Female_Und', 'Female_5 t', 'Female_10', 'Female_15', 'Female_U_1',
       'Female_18', 'Estimate;', 'Estimate_1', 'Estimate_2', 'Estimate_3',
       'Estimate_4', 'Estimate_5', 'Estimate_6', 'Estimate_7', 'Estimate_8',
       'Estimate_9', 'Estimate10', 'Estimate11', 'Female_65', 'Female_67',
       'Female_70', 'Female75 t', 'Female80 t', 'Female85 y', 'Female_Ove',
       'fullarea_age', 'ASSET_MGMT', 'LOCATION_N', 'ADDRESS', 'CITY_MUNIC',
       'COUNTY', 'STATE', 'STREET_NUM', 'STREET_NAM', 'STREET_TYP', 'ZIPCODE',
       'ASSET_SI

In [51]:
age_clip=age_intersec.copy().reset_index()

for val in age_clip:

    age_clip['weight'] = age_clip['agearea_intersec']/ age_clip['fullarea_age']
    
    age_clip['Totalpop_served'] = age_clip['weight'] * age_clip['Total_pop_']
    age_clip['Males_served'] = age_clip['weight'] * age_clip['Male']
    age_clip['Males_served_Under5'] = age_clip['weight'] * age_clip['Male_Under']
    age_clip['Males_served_Under18'] = age_clip['weight'] * age_clip['Male_Und_1']
    age_clip['Males_served_Over65'] = age_clip['weight'] * age_clip['Male_Over6']
    
    age_clip['Females_served'] = age_clip['weight'] * age_clip['Female_']
    age_clip['Females_served_Under5'] = age_clip['weight'] * age_clip['Female_Und']
    age_clip['Females_served_Under18'] = age_clip['weight'] * age_clip['Female_U_1']
    age_clip['Females_served_Over65'] = age_clip['weight'] * age_clip['Female_Ove']
        
age_clip.head()

,level_0,index,GEOID10,Id2,Total_pop_,Male,Male_Under,Male_5 to,Male_10 to,Male_15 to,...,weight,Totalpop_served,Males_served,Males_served_Under5,Males_served_Under18,Males_served_Over65,Females_served,Females_served_Under5,Females_served_Under18,Females_served_Over65
0,1,0,484530001011,484530001011,1045,515,72,35,39,30,...,0.947057,989.674200,487.734175,68.188079,166.681971,61.558682,501.940025,22.729360,90.917438,42.617549
1,5,1,484530001012,484530001012,1202,599,37,36,18,15,...,0.880910,1058.854027,527.665193,32.593676,93.376478,37.879137,531.188834,55.497341,162.087472,59.020982
2,9,2,484530001013,484530001013,2016,797,43,52,23,0,...,0.976892,1969.413526,778.582629,42.006340,115.273212,134.811045,1190.830897,66.428631,149.464419,388.802869
3,13,3,484530001021,484530001021,1578,734,42,28,90,33,...,0.079256,125.065661,58.173761,3.328744,15.296370,13.473487,66.891900,3.725023,11.412836,15.930417
4,20,4,484530001022,484530001022,1062,492,0,17,44,32,...,0.352579,374.438441,173.468656,0.000000,32.789807,39.136221,200.969785,37.020750,53.591943,32.084650


In [52]:
#income_clip.columns

In [53]:
age_calc = age_clip[['LOCATION_N','Total_pop_', 'Totalpop_served', 'Males_served','Males_served_Under5','Males_served_Under18',\
                     'Males_served_Over65','Females_served','Females_served_Under5','Females_served_Under18',\
                     'Females_served_Over65','geometry']]

age_access_data = age_calc.dissolve(by='LOCATION_N',as_index=False, aggfunc='sum')

##### Combine the park area data to the access data

In [54]:
age_access_data = age_access_data.merge(parks_, on='LOCATION_N')

In [55]:
age_access_data

,LOCATION_N,geometry,Total_pop_,Totalpop_served,Males_served,Males_served_Under5,Males_served_Under18,Males_served_Over65,Females_served,Females_served_Under5,Females_served_Under18,Females_served_Over65,ASSET_SIZE
0,A.B. Dittmar Neighborhood Park,"POLYGON ((3097425.651927478 10040361.9435798, ...",6628,605.761051,310.348052,29.288264,63.464307,21.406652,295.413000,9.954777,34.862267,27.505101,12.690000
1,Adams-Hemphill Neighborhood Park,"POLYGON ((3115627.219398506 10079404.88766938,...",16830,3294.880557,1450.604460,0.768049,31.884433,47.481905,1844.276097,44.837364,120.506000,1.645819,9.310000
2,Agave Neighborhood Park,"POLYGON ((3144849.303171726 10081396.82272123,...",4529,805.686532,396.302460,60.523242,154.058115,14.000168,409.384071,56.131810,175.106431,19.183678,16.360000
3,Alamo Pocket Park,"POLYGON ((3121188.167724669 10076733.3592437, ...",2888,115.493875,67.792269,3.019045,3.027464,2.163639,47.701606,0.004889,0.027567,7.905322,1.820000
4,Alderbrook Pocket Park,"POLYGON ((3127839.866424789 10124813.31840664,...",3241,351.309776,146.764137,7.031359,16.546967,0.000000,204.545639,14.134931,30.845836,25.977719,1.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,Wunneburger Neighborhood Park,(POLYGON ((3095156.226719216 10021252.69393206...,4954,22.156153,11.145162,0.733470,2.204882,1.220959,11.010991,0.429348,2.003625,1.314879,23.510000
287,Yates Pocket Park,"POLYGON ((3128393.558881322 10059201.38858544,...",7507,181.522781,99.063627,7.955542,20.991585,2.153803,82.459154,1.717815,16.546949,5.632885,0.933800
288,Yett Creek Neighborhood Park,"POLYGON ((3117627.349335914 10126937.73575986,...",9070,1327.392602,685.183191,37.007002,95.945945,102.407559,642.209412,50.763846,115.151781,102.118065,40.545000
289,Zilker Metro Park,(POLYGON ((3112028.695926964 10068779.90760447...,34855,14920.662889,7728.628045,398.705410,968.161330,598.071988,7192.034844,468.143437,1029.328679,702.300195,307.849066


In [56]:
age_access_data.columns

Index(['LOCATION_N', 'geometry', 'Total_pop_', 'Totalpop_served',
       'Males_served', 'Males_served_Under5', 'Males_served_Under18',
       'Males_served_Over65', 'Females_served', 'Females_served_Under5',
       'Females_served_Under18', 'Females_served_Over65', 'ASSET_SIZE'],
      dtype='object')

##### The population served by parks is nomalized by dividing the population served by the area of the park for which they are being served 

In [57]:
age_access_data['Totalpop_served_norm'] = age_access_data['Totalpop_served']/age_access_data['ASSET_SIZE']
age_access_data['Males_served_norm'] = age_access_data['Males_served']/age_access_data['ASSET_SIZE']
age_access_data['Males_served_Under5_norm'] = age_access_data['Males_served_Under5']/age_access_data['ASSET_SIZE']
age_access_data['Males_served_Under18_norm'] = age_access_data['Males_served_Under18']/age_access_data['ASSET_SIZE']
age_access_data['Males_served_Over65_norm'] = age_access_data['Males_served_Over65']/age_access_data['ASSET_SIZE']

age_access_data['Females_served_norm'] = age_access_data['Females_served']/age_access_data['ASSET_SIZE']
age_access_data['Females_served_Under5_norm'] = age_access_data['Females_served_Under5']/age_access_data['ASSET_SIZE']
age_access_data['Females_served_Under18_norm'] = age_access_data['Females_served_Under18']/age_access_data['ASSET_SIZE']
age_access_data['Females_served_Over65_norm'] = age_access_data['Females_served_Over65']/age_access_data['ASSET_SIZE']

age_access_data['Under5_served'] = age_access_data['Males_served_Under5'] + age_access_data['Females_served_Under5']
age_access_data['Under5_served_norm'] = age_access_data['Under5_served']/age_access_data['ASSET_SIZE']

age_access_data['Under18_served'] = age_access_data['Males_served_Under18'] + age_access_data['Females_served_Under18']
age_access_data['Under18_served_norm'] = age_access_data['Under18_served']/age_access_data['ASSET_SIZE']

age_access_data['Over65_served'] = age_access_data['Males_served_Over65'] + age_access_data['Females_served_Over65']
age_access_data['Over65_served_norm'] = age_access_data['Over65_served']/age_access_data['ASSET_SIZE']

In [58]:
age_access_data.to_file(wd+"data/access/access_data_age_sex_varibuff_ACS17.shp")
age_access_data

,LOCATION_N,geometry,Total_pop_,Totalpop_served,Males_served,Males_served_Under5,Males_served_Under18,Males_served_Over65,Females_served,Females_served_Under5,...,Females_served_norm,Females_served_Under5_norm,Females_served_Under18_norm,Females_served_Over65_norm,Under5_served,Under5_served_norm,Under18_served,Under18_served_norm,Over65_served,Over65_served_norm
0,A.B. Dittmar Neighborhood Park,"POLYGON ((3097425.651927478 10040361.9435798, ...",6628,605.761051,310.348052,29.288264,63.464307,21.406652,295.413000,9.954777,...,23.279196,0.784458,2.747224,2.167463,39.243041,3.092438,98.326574,7.748351,48.911752,3.854354
1,Adams-Hemphill Neighborhood Park,"POLYGON ((3115627.219398506 10079404.88766938,...",16830,3294.880557,1450.604460,0.768049,31.884433,47.481905,1844.276097,44.837364,...,198.096251,4.816043,12.943716,0.176780,45.605413,4.898541,152.390432,16.368467,49.127724,5.276877
2,Agave Neighborhood Park,"POLYGON ((3144849.303171726 10081396.82272123,...",4529,805.686532,396.302460,60.523242,154.058115,14.000168,409.384071,56.131810,...,25.023476,3.431040,10.703327,1.172596,116.655053,7.130504,329.164546,20.120082,33.183846,2.028352
3,Alamo Pocket Park,"POLYGON ((3121188.167724669 10076733.3592437, ...",2888,115.493875,67.792269,3.019045,3.027464,2.163639,47.701606,0.004889,...,26.209674,0.002686,0.015147,4.343583,3.023934,1.661502,3.055032,1.678589,10.068960,5.532396
4,Alderbrook Pocket Park,"POLYGON ((3127839.866424789 10124813.31840664,...",3241,351.309776,146.764137,7.031359,16.546967,0.000000,204.545639,14.134931,...,107.655599,7.439438,16.234650,13.672484,21.166290,11.140153,47.392802,24.943580,25.977719,13.672484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,Wunneburger Neighborhood Park,(POLYGON ((3095156.226719216 10021252.69393206...,4954,22.156153,11.145162,0.733470,2.204882,1.220959,11.010991,0.429348,...,0.468353,0.018262,0.085224,0.055928,1.162818,0.049461,4.208506,0.179009,2.535837,0.107862
287,Yates Pocket Park,"POLYGON ((3128393.558881322 10059201.38858544,...",7507,181.522781,99.063627,7.955542,20.991585,2.153803,82.459154,1.717815,...,88.304941,1.839596,17.720014,6.032218,9.673357,10.359131,37.538534,40.199757,7.786688,8.338711
288,Yett Creek Neighborhood Park,"POLYGON ((3117627.349335914 10126937.73575986,...",9070,1327.392602,685.183191,37.007002,95.945945,102.407559,642.209412,50.763846,...,15.839423,1.252037,2.840098,2.518635,87.770847,2.164776,211.097725,5.206505,204.525625,5.044411
289,Zilker Metro Park,(POLYGON ((3112028.695926964 10068779.90760447...,34855,14920.662889,7728.628045,398.705410,968.161330,598.071988,7192.034844,468.143437,...,23.362211,1.520691,3.343615,2.281313,866.848848,2.815824,1997.490008,6.488537,1300.372183,4.224058


#### A dataframe is created with the 'acess_data' geoshapefile. This dataframe is cleaned up and the values are convereted into integers. Finally, we export the dataframe into a csv file and a shapefile. 

In [59]:
age_access_data.columns

Index(['LOCATION_N', 'geometry', 'Total_pop_', 'Totalpop_served',
       'Males_served', 'Males_served_Under5', 'Males_served_Under18',
       'Males_served_Over65', 'Females_served', 'Females_served_Under5',
       'Females_served_Under18', 'Females_served_Over65', 'ASSET_SIZE',
       'Totalpop_served_norm', 'Males_served_norm', 'Males_served_Under5_norm',
       'Males_served_Under18_norm', 'Males_served_Over65_norm',
       'Females_served_norm', 'Females_served_Under5_norm',
       'Females_served_Under18_norm', 'Females_served_Over65_norm',
       'Under5_served', 'Under5_served_norm', 'Under18_served',
       'Under18_served_norm', 'Over65_served', 'Over65_served_norm'],
      dtype='object')

In [60]:
df = pd.DataFrame(age_access_data)
df['Park_Name']=df['LOCATION_N']

df['Totalpop_served'] = df['Totalpop_served'].astype(int)
df['Totalpop_served_norm'] = df['Totalpop_served_norm'].astype(int)

df['Males_served'] = df['Males_served'].astype(int)
df['Males_served_norm'] = df['Males_served_norm'].astype(int)

df['Males_served_Under5'] = df['Males_served_Under5'].astype(int)
df['Males_served_Under5_norm'] = df['Males_served_Under5_norm'].astype(int)

df['Males_served_Under18'] = df['Males_served_Under18'].astype(int)
df['Males_served_Under18_norm'] = df['Males_served_Under18_norm'].astype(int)

df['Males_served_Over65'] = df['Males_served_Over65'].astype(int)
df['Males_served_Over65_norm'] = df['Males_served_Over65_norm'].astype(int)

df['Females_served'] = df['Females_served'].astype(int)
df['Females_served_norm'] = df['Females_served_norm'].astype(int)

df['Females_served_Under5'] = df['Females_served_Under5'].astype(int)
df['Females_served_Under5_norm'] = df['Females_served_Under5_norm'].astype(int)

df['Females_served_Under18'] = df['Females_served_Under18'].astype(int)
df['Females_served_Under18_norm'] = df['Females_served_Under18_norm'].astype(int)

df['Females_served_Over65'] = df['Females_served_Over65'].astype(int)
df['Females_served_Over65_norm'] = df['Females_served_Over65_norm'].astype(int)

df['Under5_served'] = df['Under5_served'].astype(int)
df['Under5_served_norm'] = df['Under5_served_norm'].astype(int)

df['Under18_served'] = df['Under18_served'].astype(int)
df['Under18_served_norm'] = df['Under18_served_norm'].astype(int)

df['Over65_served'] = df['Over65_served'].astype(int)
df['Over65_served_norm'] = df['Over65_served_norm'].astype(int)


In [61]:
df.columns

Index(['LOCATION_N', 'geometry', 'Total_pop_', 'Totalpop_served',
       'Males_served', 'Males_served_Under5', 'Males_served_Under18',
       'Males_served_Over65', 'Females_served', 'Females_served_Under5',
       'Females_served_Under18', 'Females_served_Over65', 'ASSET_SIZE',
       'Totalpop_served_norm', 'Males_served_norm', 'Males_served_Under5_norm',
       'Males_served_Under18_norm', 'Males_served_Over65_norm',
       'Females_served_norm', 'Females_served_Under5_norm',
       'Females_served_Under18_norm', 'Females_served_Over65_norm',
       'Under5_served', 'Under5_served_norm', 'Under18_served',
       'Under18_served_norm', 'Over65_served', 'Over65_served_norm',
       'Park_Name'],
      dtype='object')

In [62]:
access_age_final = df.copy()
access_age_final = gpd.GeoDataFrame(access_age_final, geometry='geometry')
access_age_final.to_file(wd+"data/access/access_age_final_varibuff_ACS17.shp")
access_age_final.head()

,LOCATION_N,geometry,Total_pop_,Totalpop_served,Males_served,Males_served_Under5,Males_served_Under18,Males_served_Over65,Females_served,Females_served_Under5,...,Females_served_Under5_norm,Females_served_Under18_norm,Females_served_Over65_norm,Under5_served,Under5_served_norm,Under18_served,Under18_served_norm,Over65_served,Over65_served_norm,Park_Name
0,A.B. Dittmar Neighborhood Park,"POLYGON ((3097425.651927478 10040361.9435798, ...",6628,605,310,29,63,21,295,9,...,0,2,2,39,3,98,7,48,3,A.B. Dittmar Neighborhood Park
1,Adams-Hemphill Neighborhood Park,"POLYGON ((3115627.219398506 10079404.88766938,...",16830,3294,1450,0,31,47,1844,44,...,4,12,0,45,4,152,16,49,5,Adams-Hemphill Neighborhood Park
2,Agave Neighborhood Park,"POLYGON ((3144849.303171726 10081396.82272123,...",4529,805,396,60,154,14,409,56,...,3,10,1,116,7,329,20,33,2,Agave Neighborhood Park
3,Alamo Pocket Park,"POLYGON ((3121188.167724669 10076733.3592437, ...",2888,115,67,3,3,2,47,0,...,0,0,4,3,1,3,1,10,5,Alamo Pocket Park
4,Alderbrook Pocket Park,"POLYGON ((3127839.866424789 10124813.31840664,...",3241,351,146,7,16,0,204,14,...,7,16,13,21,11,47,24,25,13,Alderbrook Pocket Park


In [63]:
access_agetable = df.drop(columns=['geometry'])
access_agetable.to_csv(wd+"data/access/access_table_age_varibuff_ACS17.csv")
access_agetable.head()

,LOCATION_N,Total_pop_,Totalpop_served,Males_served,Males_served_Under5,Males_served_Under18,Males_served_Over65,Females_served,Females_served_Under5,Females_served_Under18,...,Females_served_Under5_norm,Females_served_Under18_norm,Females_served_Over65_norm,Under5_served,Under5_served_norm,Under18_served,Under18_served_norm,Over65_served,Over65_served_norm,Park_Name
0,A.B. Dittmar Neighborhood Park,6628,605,310,29,63,21,295,9,34,...,0,2,2,39,3,98,7,48,3,A.B. Dittmar Neighborhood Park
1,Adams-Hemphill Neighborhood Park,16830,3294,1450,0,31,47,1844,44,120,...,4,12,0,45,4,152,16,49,5,Adams-Hemphill Neighborhood Park
2,Agave Neighborhood Park,4529,805,396,60,154,14,409,56,175,...,3,10,1,116,7,329,20,33,2,Agave Neighborhood Park
3,Alamo Pocket Park,2888,115,67,3,3,2,47,0,0,...,0,0,4,3,1,3,1,10,5,Alamo Pocket Park
4,Alderbrook Pocket Park,3241,351,146,7,16,0,204,14,30,...,7,16,13,21,11,47,24,25,13,Alderbrook Pocket Park


In [64]:
access_age_stats = access_agetable.describe()
access_age_stats.to_csv(wd+"data/access/access_stats_age_varibuff_ACS17.csv")
access_age_stats

,Total_pop_,Totalpop_served,Males_served,Males_served_Under5,Males_served_Under18,Males_served_Over65,Females_served,Females_served_Under5,Females_served_Under18,Females_served_Over65,...,Females_served_norm,Females_served_Under5_norm,Females_served_Under18_norm,Females_served_Over65_norm,Under5_served,Under5_served_norm,Under18_served,Under18_served_norm,Over65_served,Over65_served_norm
count,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,...,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000
mean,12270.402062,3608.687285,1831.573883,117.243986,371.951890,137.759450,1776.615120,111.707904,365.295533,173.920962,...,297.470790,16.274914,57.168385,41.391753,229.405498,32.783505,737.762887,117.381443,312.140893,70.343643
std,14043.291737,7211.357128,3635.402029,239.116518,726.437688,266.665134,3582.571223,229.543672,716.642619,341.228113,...,2315.737275,126.166393,499.439992,445.375546,463.779110,242.984400,1437.739373,1048.610010,605.540167,721.158794
min,458.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4289.500000,293.500000,147.500000,7.000000,21.500000,10.000000,145.000000,7.500000,25.500000,11.500000,...,23.000000,0.000000,3.000000,2.000000,15.500000,1.000000,46.500000,7.000000,24.000000,3.000000
50%,7157.000000,724.000000,361.000000,21.000000,70.000000,23.000000,352.000000,16.000000,63.000000,32.000000,...,45.000000,2.000000,7.000000,4.000000,40.000000,4.000000,135.000000,14.000000,56.000000,8.000000
75%,14736.500000,3271.500000,1659.500000,100.500000,298.000000,103.500000,1665.500000,85.500000,264.500000,152.500000,...,92.000000,6.000000,19.000000,8.000000,191.000000,11.500000,542.500000,39.500000,239.000000,14.500000
max,112988.000000,59213.000000,29457.000000,2071.000000,4976.000000,1884.000000,29755.000000,1942.000000,4939.000000,2725.000000,...,38502.000000,2101.000000,8420.000000,7561.000000,4013.000000,4032.000000,9915.000000,17702.000000,4609.000000,12218.000000


### END OF CODE